In [4]:
import sqlite3
import pandas as pd
from collections import defaultdict, Counter


db_path = "./data/flight_database.db"
pred_path = "./results/t5_ft_baseline_with_prefix_greedy_dev.sql"      

conn = sqlite3.connect(db_path)
cur = conn.cursor()

def run_query(sql):
    try:
        cur.execute(sql)
        return True, None
    except Exception as e:
        return False, str(e)

errors = []

with open(pred_path, "r") as f:
    preds = [line.strip() for line in f if line.strip()]

for i, sql in enumerate(preds):
    ok, err = run_query(sql)
    if not ok:
        errors.append((i, sql, err))

for idx, sql, err in errors:
    print(f"[{idx}] SQL:")
    print(sql)
    print("\nError Message:")
    print(err)


[2] SQL:
SELECT DISTINCT flight_1.flight_id FROM flight flight_1, airport airport_1, airport_service airport_service_1, city city_1, airport_service airport_service_2, city city_2, days days_1, date_day date_day_1 WHERE flight_1.departure_time BETWEEN 0 AND 1200 AND( flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'BALTIMORE' AND( flight_1.arrival_time  900 AND( flight_1.arrival_time  flight_1.arrival_time AND( flight_1.flight_days = days_1.days_code AND days_1.day_name = date_day_1.day_name AND date_day_1.year = 1991 AND date_day_1.month_number = 4 AND date_day_1.day_number = 24 AND( flight_1.arrival_time  flight_1.departure_time AND 1 = 1 ) ) ) ) ) ) ) )

Error Message:
near "900": syntax error
[18] SQL:
SELECT DISTINCT fare_1.fare_id FROM fare fare_1, flight_fare flight_fare_1, flight flight_1, airport_service airport_service_1, city city_1, airport_service airport_service_2, city city_2 WHERE fare_1.ro

In [6]:
import sqlite3
from collections import defaultdict


error_categories = {
    "NO_SUCH_COLUMN": "no such column",
    "SYNTAX_ERROR": 'near ")"',
    "INCOMPLETE_INPUT": "incomplete input"
}

errors = defaultdict(list)

for i, sql in enumerate(preds):
    ok, err = run_query(sql)
    if ok:
        continue

    err_lower = err.lower()

    matched = False
    for key, phrase in error_categories.items():
        if phrase in err_lower:
            errors[key].append((i, sql, err))
            matched = True
            break

    if not matched:
        errors["OTHER"].append((i, sql, err))

print("============== ERROR SUMMARY ==============\n")
for key in error_categories:
    count = len(errors[key])
    print(f"{key}: {count}")

print(f"OTHER: {len(errors['OTHER'])}")
print("\n===========================================\n")

for key in error_categories:
    print(f"\n\n===== {key} examples =====")
    for idx, (i, sql, err) in enumerate(errors[key][:2]):
        print(f"[{i}] SQL:")
        print(sql)
        print("\nError Message:")
        print(err)



============== ERROR SUMMARY ==============

NO_SUCH_COLUMN: 20
SYNTAX_ERROR: 25
INCOMPLETE_INPUT: 4
OTHER: 25




===== NO_SUCH_COLUMN examples =====
[20] SQL:
SELECT DISTINCT flight_1.flight_id FROM flight flight_1, airport_service airport_service_1, city city_1, airport_service airport_service_2, city city_2 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'BOSTON' AND( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'SAN FRANCISCO' AND flight_1.flight_id = flight_stops_1.flight_id AND flight_stop_1.stops = 0 AND 1 = 1 )

Error Message:
no such column: flight_stops_1.flight_id
[78] SQL:
SELECT DISTINCT flight_1.flight_id FROM flight flight_1, airport_service airport_service_1, city city_1 WHERE flight_1.flight_id = flight_fare_1.flight_id AND flight_fare_1.fare_id = fare_1.fare_id AND fare_1.one_direction_cost =( SELECT 